* This notebook download raw data from WRDS and organize the data. Output of this notebook can be used by `constructed_input_price_and_cashflow.ipynb` to construct price vectors and cashflow matrices.
* A WRDS account is needed to access CRSP data through WRDS.
* Filters applied in this notebook:
    * Only includes fully taxable, non-callable, and non-flower bond issues. 
    * Certificates of deposit are excluded (ITYPE=3)
    * Remove issues whose time series of prices terminate because of ''all exchanged'' (IWHY=3)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.auto import tqdm

# Download raw data from WRDS

In [63]:
df_iss = pd.read_csv("data/tfz_iss.csv")
df_mast = pd.read_csv("data/tfz_mast.csv")
df_pay = pd.read_csv("data/tfz_pay.csv")
df_dly = pd.read_csv("data/tfz_dly.csv")

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [64]:
for df in [df_iss, df_mast, df_pay, df_dly]:
    print(df.shape)
    
df_iss.columns

(157372, 37)
(7771, 8)
(25454, 4)
(19125, 32)


Index(['KYTREASNO', 'KYCRSPID', 'CRSPID', 'TCUSIP', 'TDATDT', 'TMATDT', 'IWHY',
       'TCOUPRT', 'TNIPPY', 'TVALFC', 'TFCPDT', 'IFCPDTF', 'TFCALDT',
       'TNOTICE', 'IYMCN', 'ITYPE', 'IUNIQ', 'ITAX', 'IFLWR', 'TBANKDT',
       'TSTRIPELIG', 'TFRGNTGT', 'MCALDT', 'TMBID', 'TMASK', 'TMNOMPRC',
       'TMNOMPRC_FLG', 'TMSOURCR', 'TMACCINT', 'TMRETNUA', 'TMYLD', 'TMDURATN',
       'TMTOTOUT', 'TMPUBOUT', 'TMPCYLD', 'TMRETNXS', 'TMPDINT'],
      dtype='object')

In [65]:
df_dly["MCALDT"]

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
19120    20220630.0
19121    20220729.0
19122    20220831.0
19123    20220930.0
19124           NaN
Name: MCALDT, Length: 19125, dtype: float64

# Format raw data

## Format dataframe containing bond information

In [13]:
# change all column names to upper case
for df in [df_iss, df_mast, df_pay, df_dly]:
    df.columns = [col.upper() for col in df.columns]

# merge df_iss and df_mast into df_info, containing bond information
df_info=df_iss.join(df_mast.set_index('KYTREASNO'),on='KYTREASNO',how='outer')

# convert all date into pandas datetime format
for col in ['TDATDT','TMATDT','TFCPDT','TFCALDT','TBANKDT','TMFSTDAT',
            'TMLSTDAT','TDFSTDAT','TDLSTDAT']:
    df_info[col]=pd.to_datetime(df_info[col],format='%Y-%m-%d')
    
# calculate rounded maturity, rounded to the nearest year. This is for filtering later on. 
df_info['RoundedMaturityYears']=np.round(((df_info['TMATDT']-df_info['TDATDT']).copy()\
                                          /np.timedelta64(1, 'Y')).values)
display(df_info.head())

,KYTREASNO,KYCRSPID,CRSPID,TCUSIP,TDATDT,TMATDT,IWHY,TCOUPRT,TNIPPY,TVALFC,...,TMRETNXS,TMPDINT,TREASNOTYPE,TMFSTDAT,TMLSTDAT,TDFSTDAT,TDLSTDAT,TNAME,TREASSYM,RoundedMaturityYears
0.0,205899,19600204.4,19600204.4,0XX,NaT,1970-01-01 00:00:00.019600204,1.0,0.00,0.0,0.000,...,0.000085,0.0,1.0,1959-08-31,1960-01-29,NaT,NaT,BILL,NaN,NaN
1.0,205900,19600211.4,19600211.4,0XX,NaT,1970-01-01 00:00:00.019600211,1.0,0.00,0.0,0.000,...,0.000287,0.0,1.0,1959-08-31,1960-01-29,NaT,NaT,BILL,NaN,NaN
2.0,205901,19600215.30375,19600215.30375,0XX,1970-01-01 00:00:00.019590215,1970-01-01 00:00:00.019600215,1.0,3.75,2.0,1.875,...,0.000425,0.0,1.0,1959-02-27,1960-01-29,NaT,NaT,CTF,NaN,0.0
3.0,205902,19600215.904,19600215.904,0XX,NaT,1970-01-01 00:00:00.019600215,1.0,4.00,2.0,2.000,...,0.000539,0.0,1.0,1959-11-30,1960-01-29,NaT,NaT,NOTE,NaN,NaN
4.0,205903,19600218.4,19600218.4,0XX,NaT,1970-01-01 00:00:00.019600218,1.0,0.00,0.0,0.000,...,0.000493,0.0,1.0,1959-08-31,1960-01-29,NaT,NaT,BILL,NaN,NaN


In [39]:
sum(df_info.IWHY == 1)

125936

## Format daily prices

In [33]:
# replace missing values in df_dly
#df_dly.TDRETNUA.replace(-99,np.nan,inplace=True)
#df_dly.TDYLD.replace(-99,np.nan,inplace=True)
#df_dly.TDDURATN.replace(-1,np.nan,inplace=True)
#df_dly.TDBID.replace(0,np.nan,inplace=True)
#df_dly.TDASK.replace(0,np.nan,inplace=True)
#df_dly.TDNOMPRC.replace(0,np.nan,inplace=True)

# convert quotation date CALDT into datetime format
df_dly['MCALDT']=pd.to_datetime(df_dly['MCALDT'],format='%Y-%m-%d')

# slice df_info where KYTREASNO is in df_dly
# i.e. retain information of only securities whose prices are available
df_info_dly=df_info[df_info.KYTREASNO.isin(df_dly.KYTREASNOX.unique())]
print('Number of KYTREASNO with price data: {}\n'
      'Number of KYTREASNO with info record: {}\n'
      'Number of KYTREASNO in the intersection: {}'\
      .format(df_dly.KYTREASNOX.nunique(), len(df_info), len(df_info_dly)))

display(df_dly.head())

Number of KYTREASNO with price data: 88
Number of KYTREASNO with info record: 159305
Number of KYTREASNO in the intersection: 88


,KYTREASNOX,TIDXFAM,TTERMTYPE,TTERMLBL,MCALDT,RMTREASNO,RMTREASNO_FLG,RMCRSPID,RMCRSPID_FLG,TMBID,...,TMAVEFWD4,TMDURFWD4,TMBIDHLD1,TMASKHLD1,TMAVEHLD1,TMDURHLD1,TMBIDHLD4,TMASKHLD4,TMAVEHLD4,TMDURHLD4
0,2000001,RISKFREE,30,CRSP Risk Free Rates - 1-Month (Nominal),NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000002,RISKFREE,90,CRSP Risk Free Rates - 3-Month (Nominal),NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000003,FIXEDTERM,112,CRSP Fixed Term Index - 1-Year (Nominal),NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000004,FIXEDTERM,212,CRSP Fixed Term Index - 2-Year (Nominal),NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000005,FIXEDTERM,512,CRSP Fixed Term Index - 5-Year (Nominal),NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Format coupon payment information

In [19]:
df_pay['TPQDATE']=pd.to_datetime(df_pay['TPQDATE'],format='%Y-%m-%d')
display(df_pay.head())

,KYTREASNO,KYCRSPID,TPQDATE,PDINT
0,205901,1.960022e+07,1970-01-01 00:00:00.019600215,1.8750
1,205902,1.960022e+07,1970-01-01 00:00:00.019600215,2.0000
2,205907,1.960032e+07,1970-01-01 00:00:00.019600315,1.4375
3,205912,1.960040e+07,1970-01-01 00:00:00.019600401,0.7500
4,205920,1.960052e+07,1970-01-01 00:00:00.019600515,1.6250


# Select data

Retain: non-flower bonds, taxable bonds, bonds whose time series end because of maturity.
Read CRSP Treasury data manual for more detail. 

In [34]:
# check how time-series end
print('Unique values of IWHY: {}'.format(df_info_dly.IWHY.unique()))
# bond that are still quotes 0 (used to be np.nan)
print('number of bonds that are still quotes (IWHY=0): {}'.format((df_info_dly.IWHY==0).sum()))
#bonds that have matured (iwhy==1)
print('number of bonds that have matured (IWHY=1): {}'.format((df_info_dly.IWHY==1).sum()))
#bonds that are called for redemption (iwhy==2)
print('number of bonds that are called (IWHY=2): {}'.format((df_info_dly.IWHY==2).sum()))
#bonds that are all exchanged (iwhy==3)
print('number of bonds that are all exchanged (IWHY=3): {}'.format((df_info_dly.IWHY==3).sum()))
#Sources no longer quote issue (iwhy==4)
print('Sources no longer quote issue (IWHY=4): {}\n'.format((df_info_dly.IWHY==4).sum()))

# check types of securities
for itype in [1,2,3,4]:
    print('number of bonds with ITYPE={}: {}'.format(itype, (df_info_dly.ITYPE==itype).sum()))

Unique values of IWHY: [nan]
number of bonds that are still quotes (IWHY=0): 0
number of bonds that have matured (IWHY=1): 0
number of bonds that are called (IWHY=2): 0
number of bonds that are all exchanged (IWHY=3): 0
Sources no longer quote issue (IWHY=4): 0

number of bonds with ITYPE=1: 0
number of bonds with ITYPE=2: 0
number of bonds with ITYPE=3: 0
number of bonds with ITYPE=4: 0


In [40]:
# keep only: nonflower, taxable bonds, bonds whose time series end because of maturity.
# remove certificate of deposit (ITYPE=3)
df_info_dly_s = df_info_dly[(df_info_dly.ITYPE.isin([1,2,4]))&\
                            (df_info_dly.ITAX==1)&\
                            (df_info_dly.IFLWR==1)&(df_info_dly.IWHY!=3)]
df_dly_s = df_dly[df_dly.KYTREASNO.isin(df_info_dly_s.KYTREASNO)]
df_pay_s = df_pay[df_pay.KYTREASNO.isin(df_info_dly_s.KYTREASNO)]

# 0 means bond not matured yet
print('IWHY in selected securities: {}'.format(df_info_dly_s.IWHY.unique()))

AttributeError: 'DataFrame' object has no attribute 'KYTREASNO'

In [8]:
# maximum time to maturity in the dataset
df_info_2 = df_info_dly_s[df_info_dly_s.TDATDT>=pd.to_datetime('1961-01-01')]
max_ttm = (df_info_2['TMATDT']-df_info_2['TDATDT']).max().days
print('Max time to maturity in years: {:.2f}'.format(max_ttm/365))

Max time to maturity in years: 30.53


# Get ex-dividend mid prices

In [9]:
# B is ex-dividend
df_dly_s['B']=df_dly_s['TDNOMPRC']+df_dly_s['TDACCINT']
# Bc is cum-dividend
# df_dly_s['Bc']=df_dly_s['TDNOMPRC']+df_dly_s['TDACCINT']+df_dly_s['TDPDINT']

/tmp/ipykernel_61192/3350744440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dly_s['B']=df_dly_s['TDNOMPRC']+df_dly_s['TDACCINT']


In [10]:
# generate a T by num_security dataframe to record prices of selected securites
dly_caldt=df_dly_s.CALDT.unique()
dly_kytreasno=df_dly_s.KYTREASNO.unique()
print('Number of dates: {}\n'
      'Number of securities: {}'.format(len(dly_caldt),len(dly_kytreasno)))

df_B=pd.DataFrame(np.nan,index=dly_caldt,columns=dly_kytreasno)
pbar=tqdm(total=len(dly_kytreasno))
for kytreasno in dly_kytreasno:
    temp_caldt=df_dly_s.loc[df_dly_s.KYTREASNO==kytreasno].CALDT
    temp_tdnomprc=df_dly_s.loc[df_dly_s.KYTREASNO==kytreasno][['B']]
    df_B.loc[temp_caldt.values, kytreasno]=temp_tdnomprc.B.values
    pbar.update(1)

Number of dates: 15118
Number of securities: 5586


  0%|          | 0/5586 [00:00<?, ?it/s]

# Get lookup table between all dates and time indexes
From the earliest to the latest possible dates

In [11]:
T=len(df_B.index)
df_t_lookup=pd.DataFrame(index=df_B.index,\
                         data=np.arange(0, T),columns=['t'])
print('T: {}, min date: {}, max date: {}'.format(T, df_t_lookup.index[0],df_t_lookup.index[-1]))

# get monthly lookup table
df_t_lookup['date']=df_t_lookup.index
df_t_lookup_monthly=df_t_lookup\
    .groupby(by=[df_t_lookup.index.month, df_t_lookup.index.year]).max()\
    .reset_index()[['t','date']]\
    .set_index('date')\
    .sort_index()

df_t_lookup=df_t_lookup[['t']]

T: 15118, min date: 1961-06-14 00:00:00, max date: 2021-12-31 00:00:00


# Save processed data

In [12]:
df_B.to_pic(dir_output+'df_B.pkl')
df_info_dly_s.to_pickle(dir_output+'df_info_dly.pkl')
df_pay_s.to_pickle(dir_output+'df_pay.pkl')

df_t_lookup.to_pickle(dir_output+'df_t_lookup_daily.pkl')
df_t_lookup_monthly.to_pickle(dir_output+'df_t_lookup_monthly.pkl')# Get lookup table between dates and time indexes